In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
dataset = tfds.load("conll2003")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/14042 [00:00<?, ? examples/s]

...

Generating dev examples...:   0%|          | 0/3251 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/conll2003/conll2003/1.0.0.incompleteUU5638/conll2003-dev.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/3454 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/conll2003/conll2003/1.0.0.incompleteUU5638/conll2003-test.tfrecord*...:   …

Dataset conll2003 downloaded and prepared to /root/tensorflow_datasets/conll2003/conll2003/1.0.0. Subsequent calls will reuse this data.


In [ ]:
def preprocess_sentence(sentence):
    tokens = sentence['tokens']
    labels = sentence['ner']
    return tokens, labels

In [ ]:
def preprocess_dataset(dataset):
    preprocessed_data = []
    for sentence in dataset:
        tokens, labels = preprocess_sentence(sentence)
        preprocessed_data.append((tokens, labels))
    return preprocessed_data

In [ ]:
train_data = preprocess_dataset(dataset['train'])
test_data = preprocess_dataset(dataset['test'])

In [ ]:
for tokens, labels in train_data[:1]:
    print("Tokens:", tokens)
    print("Labels:", labels)

Tokens: tf.Tensor(
[b'"' b'If' b'they' b"'re" b'saying' b'at' b'least' b'20' b'percent' b','
 b'then' b'their' b'internal' b'forecasts' b'are' b'probably' b'saying'
 b'25' b'or' b'30' b'percent' b',' b'"' b'said' b'one' b'Sydney' b'media'
 b'analyst' b'who' b'declined' b'to' b'be' b'named' b'.'], shape=(34,), dtype=string)
Labels: tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0], shape=(34,), dtype=int64)


In [ ]:
def preprocess_data(data):
    token_sequences = []
    label_sequences = []
    for tokens, labels in data:
        token_sequences.append(tokens.numpy().tolist())
        label_sequences.append(labels.numpy().tolist())
    return token_sequences, label_sequences

train_tokens, train_labels = preprocess_data(train_data)
test_tokens, test_labels = preprocess_data(test_data)

In [ ]:
vocab = set(word for sentence in train_tokens + test_tokens for word in sentence)
word2idx = {word: idx + 1 for idx, word in enumerate(vocab)}
word2idx['<pad>'] = 0
word2idx['<unk>'] = len(word2idx)

In [ ]:
label_set = set(label for sentence_labels in train_labels + test_labels for label in sentence_labels)
label2idx = {label: idx for idx, label in enumerate(label_set)}

In [ ]:
train_tokens = [[word2idx.get(word, word2idx['<unk>']) for word in sentence] for sentence in train_tokens]
test_tokens = [[word2idx.get(word, word2idx['<unk>']) for word in sentence] for sentence in test_tokens]

In [ ]:
train_labels = [[label2idx[label] for label in labels] for labels in train_labels]
test_labels = [[label2idx[label] for label in labels] for labels in test_labels]


In [ ]:
train_tokens = tf.keras.preprocessing.sequence.pad_sequences(train_tokens, padding='post')
test_tokens = tf.keras.preprocessing.sequence.pad_sequences(test_tokens, padding='post')

In [ ]:
train_labels = tf.keras.preprocessing.sequence.pad_sequences(train_labels, padding='post')
test_labels = tf.keras.preprocessing.sequence.pad_sequences(test_labels, padding='post')

In [ ]:
model = tf.keras.Model(inputs=token_input, outputs=crf_output)


NameError: name 'token_input' is not defined

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word2idx), output_dim=50, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True)),
    tf.keras.layers.Dense(len(label2idx), activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_tokens, train_labels, epochs=5, validation_split=0.1)

Epoch 1/5
395/395 [==============================] - 73s 162ms/step - loss: 0.6541 - accuracy: 0.8443 - val_loss: 0.3583 - val_accuracy: 0.8862
Epoch 2/5
395/395 [==============================] - 60s 151ms/step - loss: 0.2337 - accuracy: 0.9300 - val_loss: 0.1915 - val_accuracy: 0.9486
Epoch 3/5
395/395 [==============================] - 59s 150ms/step - loss: 0.1006 - accuracy: 0.9751 - val_loss: 0.1395 - val_accuracy: 0.9635
Epoch 4/5
395/395 [==============================] - 58s 146ms/step - loss: 0.0489 - accuracy: 0.9891 - val_loss: 0.1316 - val_accuracy: 0.9628
Epoch 5/5
395/395 [==============================] - 59s 149ms/step - loss: 0.0297 - accuracy: 0.9935 - val_loss: 0.1250 - val_accuracy: 0.9669


In [ ]:
test_loss, test_accuracy = model.evaluate(test_tokens, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

108/108 [==============================] - 7s 32ms/step - loss: 0.2897 - accuracy: 0.9225
Test Loss: 0.2897350490093231
Test Accuracy: 0.9224938154220581


In [ ]:
pip install spacy

In [ ]:
!pip install gensim

In [ ]:
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

# Load spaCy's English NER model
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities using spaCy
def extract_named_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Function to tokenize text into sentences
def tokenize_sentences(text):
    return sent_tokenize(text)

# Function to remove stopwords from tokens
def remove_stopwords(tokens):
    stop_words = set(stopwords.words("english"))
    return [token for token in tokens if token.lower() not in stop_words]

# Function to summarize text using NLTK's frequency-based summarization
def summarize_text(text):
    sentences = tokenize_sentences(text)
    words = word_tokenize(text.lower())
    words = remove_stopwords(words)
    freq_dist = FreqDist(words)
    ranked_sentences = [(sent, sum([freq_dist[word] for word in word_tokenize(sent.lower()) if word in freq_dist])) for sent in sentences]
    ranked_sentences = sorted(ranked_sentences, key=lambda x: x[1], reverse=True)
    summary = " ".join([sent for sent, score in ranked_sentences[:3]])  # Select top 3 sentences as summary
    return summary

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

# Download NLTK data if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Load spaCy's English NER model
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities using spaCy
def extract_named_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Function to tokenize text into sentences
def tokenize_sentences(text):
    return sent_tokenize(text)

# Function to remove stopwords from tokens
def remove_stopwords(tokens):
    stop_words = set(stopwords.words("english"))
    return [token for token in tokens if token.lower() not in stop_words]

# Function to summarize text using NLTK's frequency-based summarization
def summarize_text(text):
    sentences = tokenize_sentences(text)
    words = word_tokenize(text.lower())
    words = remove_stopwords(words)
    freq_dist = FreqDist(words)
    ranked_sentences = [(sent, sum([freq_dist[word] for word in word_tokenize(sent.lower()) if word in freq_dist])) for sent in sentences]
    ranked_sentences = sorted(ranked_sentences, key=lambda x: x[1], reverse=True)
    summary = " ".join([sent for sent, score in ranked_sentences[:3]])  # Select top 3 sentences as summary
    return summary

# Example text
input_text = "Microsoft was founded by Bill Gates and Paul Allen. It is headquartered in Redmond, Washington. Microsoft develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services."

# Summarize with named entities
entities = extract_named_entities(input_text)
summary = summarize_text(input_text)

print("Summary:")
print(summary)
print("\nNamed Entities:")
print(entities)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Summary:
Microsoft develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services. It is headquartered in Redmond, Washington. Microsoft was founded by Bill Gates and Paul Allen.

Named Entities:
[('Microsoft', 'ORG'), ('Bill Gates', 'PERSON'), ('Paul Allen', 'PERSON'), ('Redmond', 'GPE'), ('Washington', 'GPE'), ('Microsoft', 'ORG')]
